In [ ]:
Write a solution to find employees who have the highest salary in each of the departments.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Employee table:
+----+-------+--------+--------------+
| id | name  | salary | departmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 70000  | 1            |
| 2  | Jim   | 90000  | 1            |
| 3  | Henry | 80000  | 2            |
| 4  | Sam   | 60000  | 2            |
| 5  | Max   | 90000  | 1            |
+----+-------+--------+--------------+
Department table:
+----+-------+
| id | name  |
+----+-------+
| 1  | IT    |
| 2  | Sales |
+----+-------+
Output: 
+------------+----------+--------+
| Department | Employee | Salary |
+------------+----------+--------+
| IT         | Jim      | 90000  |
| Sales      | Henry    | 80000  |
| IT         | Max      | 90000  |
+------------+----------+--------+
Explanation: Max and Jim both have the highest salary in the IT department and Henry has the highest salary in the Sales department.

In [113]:
import pandas as pd
data = [[1, 'Joe', 70000, 1], [2, 'Jim', 90000, 1], [3, 'Henry', 80000, 2], [4, 'Sam', 60000, 2], [5, 'Max', 90000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [114]:
employee

,id,name,salary,departmentId
0,1,Joe,70000,1
1,2,Jim,90000,1
2,3,Henry,80000,2
3,4,Sam,60000,2
4,5,Max,90000,1


In [3]:
department

,id,name
0,1,IT
1,2,Sales


In [15]:
department = department.rename(columns = {"id":"departmentId"})
department

,departmentId,name
0,1,IT
1,2,Sales


In [78]:
result = pd.merge(left = employee,right = department,on = "departmentId" , how = "inner" )
result

,id,name_x,salary,departmentId,name_y
0,1,Joe,70000,1,IT
1,2,Jim,90000,1,IT
2,5,Max,90000,1,IT
3,3,Henry,80000,2,Sales
4,4,Sam,60000,2,Sales


In [67]:
result = result.rename(columns = {"name_y":"Department","name_x": "Employee","salary":"Salary"})

In [71]:
result.groupby("Department").apply(lambda x: x[x['Salary'] == x['Salary'].max()])[["Department","Employee","Salary"]].reset_index(drop=True)

,Department,Employee,Salary
0,IT,Jim,90000
1,IT,Max,90000
2,Sales,Henry,80000


In [123]:
### Solution 1

def department_highest_salary(employee,department):
    department = department.rename(columns = {"id":"departmentId"})
    df = pd.merge(employee,department,on = "departmentId", how = "inner")
    df = df.rename(columns = {"name_y":"Department","name_x": "Employee","salary":"Salary"})    
    df = df.groupby('Department').apply(lambda x: x[x['Salary'] == x['Salary'].max()])[["Department","Employee","Salary"]].reset_index(drop=True)
    return df

department_highest_salary(employee,department)

,Department,Employee,Salary
0,IT,Jim,90000
1,IT,Max,90000
2,Sales,Henry,80000


In [130]:
### Solution 2: Using rank 

def department_highest_salary(employee,department):
    df = employee.merge(department,left_on= "departmentId",right_on="id",how = "inner")
    df['rank'] = df.groupby('name_y')['salary'].rank(method ="dense",ascending = False)
    df = df[df['rank'] == 1]
    df.rename(columns = {"name_y":"Department","name_x": "Employee","salary":"Salary"},inplace = True)
    return df [["Department","Employee","Salary"]]

department_highest_salary(employee,department)


,Department,Employee,Salary
1,IT,Jim,90000
2,IT,Max,90000
3,Sales,Henry,80000


In [142]:
### Solution 3: Using transform 
def department_highest_salary(employee,department):
    df = employee.merge(department, left_on = "departmentId", right_on = "id", how = "inner")
    max_sal = df.groupby("departmentId")['salary'].transform(max) == df['salary']
    return df[max_sal].rename(columns = {"name_y": "Department","name_x":"Employee","salary":"Salary"})[["Department","Employee","Salary"]]

department_highest_salary(employee,department)

,Department,Employee,Salary
1,IT,Jim,90000
2,IT,Max,90000
3,Sales,Henry,80000
